# Detectron2: Zebrafish

<table align="left">
<tbody><tr><td>
   <a target="_blank" href="https://colab.research.google.com/github/j0rd1smit/CS4245_cv_project_zebra_fish/blob/master/notebooks/colab/Detectron2_zebra_fish_segmatation-colab.ipynb">
     <img src="https://camo.githubusercontent.com/dfbf50eed8dd2dea5f3e0beaaf2001eeca77f314/68747470733a2f2f7777772e74656e736f72666c6f772e6f72672f696d616765732f636f6c61625f6c6f676f5f333270782e706e67" data-canonical-src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab
   </a>
 </td>
<td>
   <a target="_blank" href="https://github.com/j0rd1smit/CS4245_cv_project_zebra_fish/blob/master/notebooks/colab/Detectron2_zebra_fish_segmatation-colab.ipynb">
     <img width="32px" src="https://camo.githubusercontent.com/9a6bfd119aeed95f13553a994f2d1cd97e033768/68747470733a2f2f7777772e74656e736f72666c6f772e6f72672f696d616765732f4769744875622d4d61726b2d333270782e706e67" data-canonical-src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
 </td>
</tr></tbody></table>

## Install detectron2

In [ ]:
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!gcc --version

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [ ]:
!git clone https://github.com/j0rd1smit/CS4245_cv_project_zebra_fish.git
!cd CS4245_cv_project_zebra_fish

In [ ]:
import os
os.chdir("CS4245_cv_project_zebra_fish")

## Train on a custom dataset

In [ ]:
#General
import os
import numpy as np

# custom code
from zebrafish.configs import get_default_instance_segmentation_config, load_config
from zebrafish.dataset import register_datasets, load_all_image_in_dataset
from zebrafish.evaluation import coco_evaluation, coco_evaluation_all_model
from zebrafish.model import predict
from zebrafish.trainers import TrainingUtil
from zebrafish.visualization import plot_prediction, plot_train_vs_validation_loss, plot_segementation, plot_segementation_vs_real
from zebrafish.utils import is_in_main_dir

#detectron
from detectron2.utils.logger import setup_logger


setup_logger()

if "notebooks" in os.getcwd() and "colab" in os.getcwd():
    os.chdir("..")
    os.chdir("..")

assert is_in_main_dir(), "The notebook expectes you to be in the main directory"

register_datasets("dataset")

In [ ]:
max_iter = 1000
use_direction_classes = False

# Create a new model everytime you run this cell
cfg = get_default_instance_segmentation_config(
    use_direction_classes,
    max_iter=max_iter,
)

# Loads an old model
# cfg = load_config(path_to_model) # load cfg from disk
# use_direction_classes = cfg.DATASETS.USE_DIRECTION_CLASSES # load use_direction_classes from disk

In [ ]:
cfg.SOLVER.MAX_ITER = max_iter # Increase the number of iters here if needed.

In [ ]:
resume = True # Set to true if want to continue training where use left off.
TrainingUtil.train(cfg, resume=resume)

## Learning curve

In [ ]:
plot_train_vs_validation_loss([cfg.OUTPUT_DIR], show_training_loss=True)

## Evaluation

In [ ]:
coco_evaluation(cfg, dataset_name="test")


## Evaluation against other models

In [ ]:
bbox_df, segm_df = coco_evaluation_all_model("test")

In [ ]:
bbox_df

In [ ]:
segm_df

## Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

## Visualization
Here we display the prediction for the unseen test set.

In [ ]:
threshold = 0.75
n_images = 10
dataset = "test"

images = load_all_image_in_dataset(dataset, cfg)[:n_images]
predictions = predict(cfg, images, threshold=threshold)

### Prediction

In [ ]:
for image, prediction in zip(images, predictions):
    plot_prediction(image, prediction, cfg)
    plot_segementation(image, prediction)
    plot_segementation_vs_real(image, prediction)